In [8]:
import gzip

path = "C:/Users/me/news.txt.gz"  

texts = []
labels = []

with gzip.open(path, 'rt', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) != 3:
            continue
        topic, title, text = parts
        labels.append(topic)
        texts.append(text)

print("Количество документов:", len(texts))
print("Пример темы:", labels[0])
print("Пример текста:", texts[0][:200], "...")


Количество документов: 10000
Пример темы: style
Пример текста: Парусная гонка Giraglia Rolex Cup пройдет в Средиземном море в 64-й раз. Победители соревнования, проводимого с 1953 года Yacht Club Italiano, помимо других призов традиционно получают в подарок часы  ...


In [9]:
!pip install natasha razdel pymorphy2 nltk gensim



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
    --------------------------------------- 0.5/24.4 MB 3.5 MB/s eta 0:00:07
   -- ------------------------------------- 1.6/24.4 MB 4.5 MB/s eta 0:00:06
   ----- ---------------------------------- 3.4/24.4 MB 6.4 MB/s eta 0:00:04
   -------- ------------------------------- 5.2/24.4 MB 7.1 MB/s eta 0:00:03
   ------------ --------------------------- 7.3/24.4 MB 7.7 MB/s eta 0:00:03
   --------------- ------------------------ 9.4/24.4 MB 8.1 MB/s eta 0:00:02
   ------------------ --------------------- 11.3/24.4 MB 8.3 MB/s eta 0:00:02
   --------------------- ------------------ 13.1/24.4 MB 8.3 MB/s eta 0:00:02
   ------------------------ --------------- 14.9/24.4 MB 8.2 MB/s eta 0:00:02
   --------------------------- ------------ 16.5/24.4 MB 8.1 MB/s eta 0:00:01
   ----------------------------- ---------- 18.1/24.4 MB 8.0 MB/s eta 0:00:01
   -------------------------------- ------- 19.7/24.4 MB 8.0 MB/s eta 0:00:01

In [10]:
from razdel import tokenize
from natasha import MorphVocab, NewsMorphTagger, NewsEmbedding, Doc, Segmenter
import pymorphy2
import nltk
from nltk.corpus import stopwords
import re

# Загружаем стоп-слова
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))

# Лемматизатор
morph = pymorphy2.MorphAnalyzer()

# Функция очистки текста
def preprocess_text(text):
    # 1. Удаляем все, кроме букв
    text = re.sub(r'[^а-яА-ЯёЁ ]', ' ', text)
    text = text.lower()

    # 2. Токенизация
    tokens = [t.text for t in tokenize(text)]

    # 3. Лемматизация + удаление стоп-слов
    lemmas = []
    for t in tokens:
        if t in russian_stopwords:
            continue
        p = morph.parse(t)[0].normal_form
        if len(p) > 2:            # удаляем слишком короткие слова
            lemmas.append(p)

    return lemmas


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\me\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
C:\Users\me\anaconda3\Lib\site-packages\pymorphy2\analyzer.py:114: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [11]:
%%time

processed_texts = [preprocess_text(t) for t in texts]

print("Готово!")
print("Пример исходного текста:\n", texts[0][:200], "...")
print("\nПосле обработки:\n", processed_texts[0][:20])


Готово!
Пример исходного текста:
 Парусная гонка Giraglia Rolex Cup пройдет в Средиземном море в 64-й раз. Победители соревнования, проводимого с 1953 года Yacht Club Italiano, помимо других призов традиционно получают в подарок часы  ...

После обработки:
 ['парусный', 'гонка', 'пройти', 'средиземный', 'море', 'победитель', 'соревнование', 'проводить', 'год', 'помимо', 'другой', 'приз', 'традиционно', 'получать', 'подарок', 'часы', 'швейцарский', 'бренд', 'сообщаться', 'пресс']
CPU times: total: 6min 32s
Wall time: 6min 44s


In [12]:
clean_texts = []
clean_labels = []

for text, label in zip(processed_texts, labels):
    if len(text) > 0:
        clean_texts.append(text)
        clean_labels.append(label)

print("Оставлено документов:", len(clean_texts))


Оставлено документов: 9999


In [13]:
from gensim.models import Word2Vec

# размерность вектора слова
VECTOR_SIZE = 200  

# Обучаем модель Word2Vec на предобработанных текстах
w2v_model = Word2Vec(
    sentences=clean_texts,   # список списков лемм
    vector_size=VECTOR_SIZE,
    window=5,
    min_count=5,      # слова реже 5 раз игнорируются
    workers=4,
    sg=1,             # 1 = skip-gram, 0 = CBOW
    epochs=10
)

print("Размер словаря:", len(w2v_model.wv))


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


Размер словаря: 17836


In [14]:
# Проверим похожие слова для какого-нибудь термина, если он есть в словаре
word = "футбол"
if word in w2v_model.wv:
    print("Слова, похожие на:", word)
    for w, sim in w2v_model.wv.most_similar(word, topn=10):
        print(w, "->", sim)
else:
    print(f"Слова '{word}' нет в словаре.")


Слова, похожие на: футбол
хоккей -> 0.6058688163757324
баскетбол -> 0.5930256843566895
черчёс -> 0.5888170003890991
тарпищев -> 0.5857967138290405
фифа -> 0.5799240469932556
юношеский -> 0.5776334404945374
чемпионат -> 0.5741127729415894
мундиаль -> 0.5739104747772217
дзюдо -> 0.5661994218826294
детско -> 0.5641145706176758


In [15]:
import numpy as np

def document_vector_mean(tokens, model):
    """
    Преобразует документ (список лемм) в один вектор,
    усредняя векторы всех слов, которые есть в словаре Word2Vec.
    """
    vectors = [model.wv[w] for w in tokens if w in model.wv]
    if not vectors:
        # если ни одного слова нет в словаре — возвращаем нулевой вектор
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)


In [16]:
X_mean = np.vstack([document_vector_mean(doc, w2v_model) for doc in clean_texts])
y = np.array(clean_labels)

print("Форма матрицы признаков:", X_mean.shape)


Форма матрицы признаков: (9999, 200)


In [17]:
from sklearn.model_selection import train_test_split

X_train_mean, X_test_mean, y_train, y_test = train_test_split(
    X_mean,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y  # сохраняем пропорции классов
)

print("Train:", X_train_mean.shape, "Test:", X_test_mean.shape)


Train: (7999, 200) Test: (2000, 200)


In [18]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Линейное ядро — стандартный вариант для задач классификации текстов
svm_mean = SVC(kernel='linear', C=1.0, random_state=42)

svm_mean.fit(X_train_mean, y_train)
y_pred_mean = svm_mean.predict(X_test_mean)

print("Точность (accuracy), усреднение векторов слов:", accuracy_score(y_test, y_pred_mean))
print("\nОтчёт по классам:")
print(classification_report(y_test, y_pred_mean))


Точность (accuracy), усреднение векторов слов: 0.8405

Отчёт по классам:
              precision    recall  f1-score   support

    business       0.68      0.35      0.46        72
     culture       0.88      0.93      0.90       279
   economics       0.78      0.91      0.84       275
      forces       0.80      0.87      0.83       154
        life       0.80      0.82      0.81       273
       media       0.82      0.80      0.81       295
     science       0.86      0.76      0.80       286
       sport       0.96      0.97      0.96       288
       style       0.89      0.82      0.85        39
      travel       0.78      0.64      0.70        39

    accuracy                           0.84      2000
   macro avg       0.82      0.79      0.80      2000
weighted avg       0.84      0.84      0.84      2000



In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Преобразуем токенизированные документы обратно в строки
docs_for_tfidf = [" ".join(doc) for doc in clean_texts]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(docs_for_tfidf)

feature_names = tfidf_vectorizer.get_feature_names_out()

print("Форма TF-IDF матрицы:", tfidf_matrix.shape)


Форма TF-IDF матрицы: (9999, 52978)


In [20]:
def document_vector_tfidf(doc_index, model, tfidf_matrix, feature_names):
    """
    Строит вектор документа как среднее взвешенное векторов слов
    с весами TF-IDF.
    """
    row = tfidf_matrix[doc_index]
    indices = row.indices     # индексы слов
    data = row.data           # веса TF-IDF

    vectors = []
    weights = []

    for idx, weight in zip(indices, data):
        word = feature_names[idx]
        if word in model.wv:
            vectors.append(model.wv[word] * weight)
            weights.append(weight)

    if not vectors:
        return np.zeros(model.vector_size)

    vectors = np.vstack(vectors)
    weights = np.array(weights)

    # среднее взвешенное
    return np.sum(vectors * weights[:, None], axis=0) / np.sum(weights)


In [21]:
# Строим матрицу признаков
X_tfidf = np.vstack([
    document_vector_tfidf(i, w2v_model, tfidf_matrix, feature_names)
    for i in range(len(clean_texts))
])

print("Форма матрицы признаков (TF-IDF + Word2Vec):", X_tfidf.shape)

# Разбиение на train/test
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(
    X_tfidf,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Обучаем SVM на новой матрице
svm_tfidf = SVC(kernel='linear', C=1.0, random_state=42)
svm_tfidf.fit(X_train_tfidf, y_train_tfidf)
y_pred_tfidf = svm_tfidf.predict(X_test_tfidf)

print("Точность (accuracy), TF-IDF + Word2Vec:", accuracy_score(y_test_tfidf, y_pred_tfidf))
print("\nОтчёт по классам (TF-IDF + Word2Vec):")
print(classification_report(y_test_tfidf, y_pred_tfidf))


Форма матрицы признаков (TF-IDF + Word2Vec): (9999, 200)
Точность (accuracy), TF-IDF + Word2Vec: 0.765

Отчёт по классам (TF-IDF + Word2Vec):
              precision    recall  f1-score   support

    business       0.00      0.00      0.00        72
     culture       0.79      0.89      0.83       279
   economics       0.71      0.91      0.79       275
      forces       0.70      0.82      0.75       154
        life       0.64      0.79      0.71       273
       media       0.75      0.74      0.75       295
     science       0.83      0.69      0.75       286
       sport       0.96      0.94      0.95       288
       style       0.89      0.21      0.33        39
      travel       1.00      0.03      0.05        39

    accuracy                           0.77      2000
   macro avg       0.73      0.60      0.59      2000
weighted avg       0.75      0.77      0.74      2000



C:\Users\me\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\me\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\me\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
